This notebook demonstrates sentiment analysis using LSTM.

Original source code: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb

In [112]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from torchtext import datasets
import itertools
import time
import os

In [7]:
!python -m spacy download en


    Linking successful
    /Users/anton/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
    /Users/anton/anaconda3/lib/python3.7/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



When we use `include_lengths`, this produces a tuple (first element - the text, second element - its length). This allows usage of packed padded sequences.

In [10]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [115]:
DATA_PATH = os.path.expanduser(r"~/pytorch_data")
DATA_PATH

'/Users/anton/pytorch_data'

In [16]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root=DATA_PATH)

In [20]:
len(train_data), len(test_data)

(25000, 25000)

In [21]:
train_data, valid_data = train_data.split()

In [22]:
len(train_data), len(valid_data), len(test_data)

(17500, 7500, 25000)

In [82]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 vectors_cache = os.path.join(DATA_PATH, "vector_cache"))

LABEL.build_vocab(train_data)

In [83]:
len(TEXT.vocab)

25002

In [84]:
TEXT.vocab.vectors.shape

torch.Size([25002, 100])

In [85]:
TEXT.vocab.vectors[TEXT.vocab.stoi[TEXT.unk_token]]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [86]:
TEXT.vocab.vectors[TEXT.vocab.stoi[TEXT.pad_token]]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [87]:
', '.join(itertools.islice(TEXT.vocab.stoi.keys(), 100))

'<unk>, <pad>, the, ,, ., a, and, of, to, is, in, I, it, that, ", \'s, this, -, /><br, was, as, with, movie, for, film, The, but, on, (, n\'t, ), you, are, not, have, his, be, he, one, !, at, by, all, who, an, they, from, like, her, so, \', about, has, or, It, out, just, do, ?, some, good, more, would, very, up, what, This, there, time, can, when, which, she, had, only, story, really, if, were, see, their, even, no, my, did, me, does, than, ..., much, :, get, been, could, into, well, we, will, people, bad'

In [88]:
uppercase_words = [key for key in TEXT.vocab.stoi.keys() if any([c for c in key if c.isupper()])]
len(uppercase_words), ', '.join(itertools.islice(uppercase_words, 100))

(9076,
 'I, The, It, This, />The, But, And, A, He, There, In, If, />I, They, You, What, She, As, TV, That, DVD, When, American, />This, So, John, Hollywood, One, We, For, Not, However, My, />It, All, No, Even, After, THE, At, Well, To, His, Michael, New, Do, Then, How, Why, While, Also, Of, Some, Just, James, English, David, Mr., />There, With, Now, />In, Robert, />If, British, Man, Richard, Oh, Paul, Although, Oscar, />But, York, George, Jack, OK, On, King, First, God, Peter, Maybe, Unfortunately, Disney, B, Japanese, Joe, Her, Lee, />A, />And, Who, Yes, Is, America, From, />As, Tom, Its, French')

In [89]:
LABEL.vocab.stoi

defaultdict(None, {'pos': 0, 'neg': 1})

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [91]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [102]:
class ModelWithLSTM(nn.Module):
    def __init__(self, embeddings, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(embeddings, padding_idx=pad_idx, freeze=False)
        
        self.rnn = nn.LSTM(embeddings.shape[1], 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        num_directions = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * num_directions, output_dim)

        self.dropout = nn.Dropout(dropout)
        
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
#                  bidirectional, dropout, pad_idx):
#         super().__init__()
        
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
#         self.rnn = nn.LSTM(embedding_dim, 
#                            hidden_dim, 
#                            num_layers=n_layers, 
#                            bidirectional=bidirectional, 
#                            dropout=dropout)
        
#         num_directions = 2 if bidirectional else 1
#         self.fc = nn.Linear(hidden_dim * num_directions, output_dim)

#         self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = [sentence len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        # embedded = [sentence len, batch size, emb dim]
        
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        _, (hidden, _) = self.rnn(packed_embedded)

        # hidden = [num layers * num directions, batch size, hid dim]
        
        # Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        # hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)

In [103]:
model = ModelWithLSTM(
    TEXT.vocab.vectors,
    hidden_dim = 256,
    output_dim = 1,
    n_layers = 2, 
    bidirectional = True,
    dropout = 0.5,
    pad_idx = TEXT.vocab.stoi[TEXT.pad_token])

In [104]:
print("Total number of params in model:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Total number of params in model: 4810857


In [105]:
model.embedding.weight.data

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.2194, -0.4329,  0.1889,  ..., -0.5509,  0.0359,  0.8901],
        [-0.1451,  0.4990, -0.4001,  ...,  0.2153,  0.1783,  0.3516],
        [-0.2197,  0.5010, -0.0146,  ...,  0.0834, -0.1279,  0.4475]])

In [107]:
optimizer = optim.Adam(model.parameters())

In [108]:
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [109]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    return correct.sum() / len(correct)

In [120]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [111]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [113]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [121]:
N_EPOCHS = 5
MODEL_PATH = os.path.join(DATA_PATH, "sentiment_lstm_best.pt")

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_PATH)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

tensor([[ 263,  162,   11,  ...,    0,   11,   11],
        [   5, 1584,   34,  ...,    0, 1097,  201],
        [1115,    8,  127,  ..., 3445,    0,   16],
        ...,
        [  16,   82,    6,  ...,    1,    1,    1],
        [2801, 5952,  205,  ...,    1,    1,    1],
        [   4,    4,    4,  ...,    1,    1,    1]]) tensor(0.6324, grad_fn=<BinaryCrossEntropyWithLogitsBackward>) tensor(0.6250)
tensor([[  25,   11,   11,  ...,   54,   11,  162],
        [  22,   34, 6973,  ...,   15, 1071, 6384],
        [   6,    8,   55,  ...,  279,   64,    3],
        ...,
        [ 424,   60, 6198,  ...,    1,    1,    1],
        [ 825,   38, 3579,  ...,    1,    1,    1],
        [   4,    4,    4,  ...,    1,    1,    1]]) tensor(0.6382, grad_fn=<BinaryCrossEntropyWithLogitsBackward>) tensor(0.6250)
tensor([[   66,   154,    16,  ..., 16170,    11,   468],
        [   14,    32,    22,  ...,   210,   246,   347],
        [    0,   414,     9,  ...,  6080,    16,     2],
        ...,
     

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load(MODEL_PATH))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')